# MICROSOFT COMPUTER VISION API - DEMO

## Intro

Before running this notebook please:

1. Create a Cognitive Services resource using the Azure portal.

2. Configure the environment variables for authentication:

https://docs.microsoft.com/azure/cognitive-services/cognitive-services-apis-create-account?tabs=multiservice%2Cunix#configure-an-environment-variable-for-authentication

COMPUTER_VISION_SUBSCRIPTION_KEY
COMPUTER_VISION_ENDPOINT


................. for macOS

export COMPUTER_VISION_SUBSCRIPTION_KEY=*********

export COMPUTER_VISION_ENDPOINT=*********

source .bash_profile



-------------------------------
Computer Vision API - v2.0 - documentation:

https://westcentralus.dev.cognitive.microsoft.com/docs/services/5adf991815e1060e6355ad44/operations/56f91f2e778daf14a499e1fa

https://docs.microsoft.com/pl-pl/azure/cognitive-services/Computer-vision/quickstarts/python-hand-text


WEB API: https://azure.microsoft.com/en-us/services/cognitive-services/computer-vision/


DEMO: https://github.com/microsoft/Cognitive-Vision-Python/blob/master/Jupyter%20Notebook/Computer%20Vision%20API%20Example.ipynb


## Load libraries

In [ ]:
#! pip install opencv-python

In [ ]:
import requests
#import cv2
import operator
import numpy as np

%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

from PIL import Image
from io import BytesIO
import os
import sys


import warnings
warnings.simplefilter("ignore", DeprecationWarning)

## Add subscription key and endpoint to your variables.

In [ ]:
# Add your Computer Vision subscription key and endpoint to your environment variables.
if 'COMPUTER_VISION_SUBSCRIPTION_KEY' in os.environ:
    SUBSCRIPTION_KEY = os.environ['COMPUTER_VISION_SUBSCRIPTION_KEY']
else:
    print("\nSet the COMPUTER_VISION_SUBSCRIPTION_KEY environment variable.\n**Restart your shell or IDE for changes to take effect.**")
    sys.exit()

if 'COMPUTER_VISION_ENDPOINT' in os.environ:
    ENDPOINT = os.environ['COMPUTER_VISION_ENDPOINT']
    

## Define functions

### Local image

In [ ]:
def SendRequestLocal(image_path, 
                     params = {'visualFeatures': 'Categories, Description, Color, Faces'},
                     subscription_key = SUBSCRIPTION_KEY,
                     endpoint = ENDPOINT):

    #Read the image into a byte array
    image_data = open(image_path, "rb").read()

    
    analyze_url = endpoint + "vision/v2.0/analyze"

    headers = {'Ocp-Apim-Subscription-Key': subscription_key,
               'Content-Type': 'application/octet-stream'}

    #params = {'visualFeatures': 'Categories, Description, Color, Faces'}

    response = requests.post(analyze_url,
                             headers=headers,
                             params=params,
                             data=image_data)

    response.raise_for_status()
    
    analysis = response.json()
    
    return analysis




def PlotImageLocal(image_path,
                   figsize=(15, 20)):

    
    img = mpimg.imread(image_path)


    ig, ax = plt.subplots(figsize=figsize)
    fig = ax.imshow( img )
    
    return fig

### Remote image (from URL)

In [ ]:
def SendRequestUrl(image_url, 
                   params = {'visualFeatures': 'Categories, Description'},
                   subscription_key = SUBSCRIPTION_KEY,
                   endpoint = ENDPOINT):

    
    analyze_url = endpoint + "vision/v2.0/analyze"

    headers = {'Ocp-Apim-Subscription-Key': subscription_key}

    response = requests.post(analyze_url,
                             headers=headers,
                             params=params,
                             json={'url': image_url})

    response.raise_for_status()
    
    analysis = response.json()
    
    return analysis




def PlotImageUrl(image_url,
                 figsize=(15, 20)):
    
    image = Image.open(BytesIO(requests.get(image_url).content))
    
    ig, ax = plt.subplots(figsize=figsize)
    fig = ax.imshow( image )
    
    return fig

### Text Analysis API

In [ ]:
def SendRequestOCRUrl(image_url, 
                     subscription_key = SUBSCRIPTION_KEY,
                     endpoint = ENDPOINT):

    text_url = endpoint + "vision/v2.0/read/core/asyncBatchAnalyze"

    headers = {'Ocp-Apim-Subscription-Key': subscription_key}

    data = {'url': image_url}
    
    response = requests.post(text_url,
                             headers=headers,
                             json=data)

    response.raise_for_status()
    


    # Extracting text requires two API calls: One call to submit the
    # image for processing, the other to retrieve the text found in the image.

    # Holds the URI used to retrieve the recognized text.
    operation_url = response.headers["Operation-Location"]

    # The recognized text isn't immediately available, so poll to wait for completion.
    analysis = {}
    poll = True
    while (poll):
        response_final = requests.get(
            response.headers["Operation-Location"], headers=headers)
        analysis = response_final.json()
        print(analysis)
        time.sleep(1)
        if ("recognitionResults" in analysis):
            poll = False
        if ("status" in analysis and analysis['status'] == 'Failed'):
            poll = False
    
    return analysis


### Print Rectangles

In [ ]:
## Print rectangles and descriptions

def PrintRectFace(rectangles,
                    fig,
                    color="fuchsia",
                    linewidth=2):
    for face in rectangles:
        rec = plt.Rectangle(xy=(face["faceRectangle"]["left"], face["faceRectangle"]["top"]),
            width=face["faceRectangle"]["width"], height=face["faceRectangle"]["height"],
                fill=False, edgecolor=color, linewidth=linewidth)

        fig.axes.add_patch(rec)
        print (face)
    return fig




def PrintRectObject(rectangles,
                    fig,
                    color="fuchsia",
                    linewidth=2):
    for obj in rectangles:
        rec = plt.Rectangle(xy=(obj["rectangle"]["x"], obj["rectangle"]["y"]),
            width=obj["rectangle"]["w"], height=obj["rectangle"]["h"],
                fill=False, edgecolor=color, linewidth=linewidth)

        fig.axes.add_patch(rec)
        print (obj)
    return fig


# FACE & CELEBRITY DETECTION - local

## Example #1

In [ ]:
##  Set image_path to the local path of an image that you want to analyze.
image_path = "/Users/doma022/Desktop/AI_Workshop/T_Rabe2.jpg"



# Send request to the CV API
analysis = SendRequestLocal(image_path,
                            params = {'visualFeatures': 'Categories, Description, Faces',
                                      'details': 'Celebrities'})



## Print the image and the API response

fig = PlotImageLocal(image_path)

## Sort faces from left to right
sorted_faces = sorted(analysis["faces"], key=lambda k: k['faceRectangle']["left"]) 

## Print rectangles and descriptions
PrintRectFace(sorted_faces, fig)

#print(analysis)
#print(analysis["description"]["captions"])

print("======")
print("CAPTION:")
image_caption = analysis["description"]["captions"][0]["text"].capitalize()
print(image_caption)


## CELEBRITY DETECTION

In [ ]:
## Print the image and the API response

fig = PlotImageLocal(image_path)

## Sort faces from left to right
sorted_celeb= sorted(analysis["categories"][0]["detail"]["celebrities"],
                     key=lambda k: k['faceRectangle']["left"]) 

## Print rectangles and descriptions
PrintRectFace(sorted_celeb, fig, color='red')


print("======")
print("CAPTION:")
image_caption = analysis["description"]["captions"][0]["text"].capitalize()
print(image_caption)


In [ ]:
analysis["description"]["tags"]

## Example #2

In [ ]:
##  Set image_path to the local path of an image that you want to analyze.
image_path = "/Users/doma022/Desktop/AI_Workshop/_DSC7162.jpg"


# Send request to the CV API
analysis = SendRequestLocal(image_path,
                            params = {'visualFeatures': 'Categories, Description, Faces',
                                      'details': 'Celebrities'})



## Print the image and the API response

fig = PlotImageLocal(image_path)


print("FACES:")

## Sort faces from left to right
sorted_faces = sorted(analysis["faces"], key=lambda k: k['faceRectangle']["left"]) 

## Print rectangles and descriptions
PrintRectFace(sorted_faces, fig, color="yellow")


print("======")
print("CAPTION:")
image_caption = analysis["description"]["captions"][0]["text"].capitalize()
print(image_caption)


In [ ]:
analysis["description"]["tags"]

# OBJECTS - remote

## Example #3

In [ ]:
# Cologne
image_path = "/Users/doma022/Desktop/AI_Workshop/_DSC6786.jpg"

# Send request to the CV API
analysis = SendRequestLocal(image_path,
                            params = {'visualFeatures': 'Categories, Description, Objects'})


## Print the image and the API response

fig = PlotImageLocal(image_path)


print("OBJECTS:")
## Sort faces from left to right
sorted_obj = sorted(analysis["objects"], key=lambda k: k['rectangle']["x"]) 

## Print rectangles and descriptions
PrintRectObject(sorted_obj, fig)



print("======")
print("CAPTION:")
image_caption = analysis["description"]["captions"][0]["text"].capitalize()
print(image_caption)


In [ ]:
analysis["description"]["tags"]

## Example #4

In [ ]:
# PRH
image_path = "https://i.ytimg.com/vi/T3kXO_SzeTQ/maxresdefault.jpg"


# Send request to the CV API
analysis = SendRequestUrl(image_path,
                          params = {'visualFeatures': 'Categories, Description, Objects, Faces',
                                    'details': 'Celebrities'})


## Print the image and the API response

fig = PlotImageUrl(image_path)



print("OBJECTS:")

## Sort faces from left to right
sorted_obj = sorted(analysis["objects"], key=lambda k: k['rectangle']["x"]) 

## Print rectangles and descriptions
PrintRectObject(sorted_obj, fig)



print("======")
print("CAPTION:")
image_caption = analysis["description"]["captions"][0]["text"].capitalize()
print(image_caption)


In [ ]:
analysis

# BRANDS

## Example #5

In [ ]:
# Brands
image_path = "/Users/doma022/Desktop/AI_Workshop/brands.jpg"

# Send request to the CV API
analysis = SendRequestLocal(image_path,
                            params = {'visualFeatures': 'Categories, Description, Objects, Brands'})


## Print the image and the API response

fig = PlotImageLocal(image_path)

print("======")
print("CAPTION:")
image_caption = analysis["description"]["captions"][0]["text"].capitalize()
print(image_caption)


In [ ]:
analysis

## Example #6

In [ ]:
# Cologne
image_path = "/Users/doma022/Desktop/AI_Workshop/reebook.jpg"

# Send request to the CV API
analysis = SendRequestLocal(image_path,
                            params = {'visualFeatures': 'Categories, Description, Objects, Brands'})


## Print the image and the API response

fig = PlotImageLocal(image_path)



print("OBJECTS:")
## Sort faces from left to right
sorted_obj = sorted(analysis["objects"], key=lambda k: k['rectangle']["x"]) 

## Print rectangles and descriptions
PrintRectObject(sorted_obj, fig)


print("======")
print("BRANDS:")
## Sort faces from left to right
sorted_brands = sorted(analysis["brands"], key=lambda k: k['rectangle']["x"]) 

## Print rectangles and descriptions
PrintRectObject(sorted_brands, fig, color="blue")


#print(analysis)
#print(analysis["description"]["captions"])

print("======")
print("CAPTION:")
image_caption = analysis["description"]["captions"][0]["text"].capitalize()
print(image_caption)


# OCR (optical character recognition)

## Example #7

In [ ]:
#image_path = "/Users/doma022/Desktop/AI_Workshop/OCR.jpg"
image_path = "https://media.npr.org/assets/img/2016/04/17/handwritten-note_wide-941ca37f3638dca912c8b9efda05ee9fefbf3147.jpg"

# Display the image and overlay it with the extracted text.
fig = PlotImageUrl(image_path)

In [ ]:
image_path = "https://media.npr.org/assets/img/2016/04/17/handwritten-note_wide-941ca37f3638dca912c8b9efda05ee9fefbf3147.jpg"


# Send request to the CV API
analysis = SendRequestOCRUrl(image_path)


# Display the image and overlay it with the extracted text.
fig = PlotImageUrl(image_path)



polygons = []
if ("recognitionResults" in analysis):
    # Extract the recognized text, with bounding boxes.
    polygons = [(line["boundingBox"], line["text"])
                for line in analysis["recognitionResults"][0]["lines"]]

    
for polygon in polygons:
    vertices = [(polygon[0][i], polygon[0][i+1])
                for i in range(0, len(polygon[0]), 2)]
    text = polygon[1]
    patch = Polygon(vertices, closed=True, fill=False, linewidth=2, color='y')
    fig.axes.add_patch(patch)
    plt.text(vertices[0][0], vertices[0][1], text, fontsize=20, va="top", color='red')
